## Загрузка датасета

In [1]:
import pandas as pd
import numpy as np

In [2]:
FILE_TYPE = 'csv'
df_original = []
if FILE_TYPE == 'xlsx':
    HABR_DATASET_PATH = 'habr.xlsx'
    df_original = pd.read_excel(HABR_DATASET_PATH)
elif FILE_TYPE == 'csv':
    HABR_DATASET_PATH = 'vacancies_data.csv'
    df_original = pd.read_csv(HABR_DATASET_PATH, encoding='utf8')

In [3]:
df_original

,Ссылка на объявление,Дата размещения,Название,Зарплата от,Зарплата до,Валюта,Вакансия,Стаж,Требования,Город,Тип занятости,Можно удаленно,Описание,Работодатель,Ссылка на работодателя,Рейтинг,Офисы,Число сотрудников,Дата сбора данных
0,https://career.habr.com/vacancies/1000144781,2024-06-27 16:50:48,Архитектор серверной инфраструктуры,NaN,500000.0,₽,Архитектор программного обеспечения,Старший (Senior),JavaScript / Node.js / Python / Elasticsearch ...,Москва,Полный рабочий день,NaN,~~~ О компании и команде\nО нас: Амбициозная к...,Ваша кадровая служба,https://vks.agency,NaN,"Россия, Москва, ул. Атарбекова, д. 4",0.0,2024-07-01 23:06:03
1,https://career.habr.com/vacancies/1000139937,2024-06-28 10:40:49,Руководитель группы FI/CO/FM,NaN,NaN,NaN,Менеджер технической поддержки,NaN,SAP / ABAP,Москва,Полный рабочий день,True,~~~ О компании и команде\nМеждународная сеть м...,Fix Price,https://fix-price.ru/,NaN,"Россия, Москва, Щепкина, д.33",2.0,2024-07-01 23:06:03
2,https://career.habr.com/vacancies/1000143735,2024-06-04 16:24:59,Intern Project Manager,5000.0,25000.0,₽,Менеджер проекта,Стажёр (Intern),Управление проектами / Agile / Управление риск...,"Москва, Санкт-Петербург, Казань",Полный рабочий день,True,~~~ О компании и команде\nКомпания FedAG - быс...,FedAG,https://career.habr.com/companies/fedag,4.90,"Россия, Санкт-Петербург, Институтский переулок 5",1.0,2024-07-01 23:06:04
3,https://career.habr.com/vacancies/1000143869,2024-07-01 12:09:30,"Middle+ Backend (Node.js, TypeScript) Developer",210000.0,NaN,₽,Бэкенд разработчик,Средний (Middle),Node.js / TypeScript / MongoDB / Redis,NaN,Полный рабочий день,True,~~~ О компании и команде\nОптимакрос – крупный...,Optimacros,https://optimacros.com,NaN,"Россия, Москва, Дмитровское шоссе, 81",0.0,2024-07-01 23:06:04
4,https://career.habr.com/vacancies/1000144076,2024-06-25 19:33:45,Системный аналитик,NaN,NaN,NaN,Системный аналитик,NaN,SQL / REST,NaN,NaN,True,~~~ О компании и команде\nОбязанности: Наша ко...,IBS,http://www.ibs.ru/,4.38,"Россия, Омск, ул. Маяковского, 74А\nРоссия, Ша...",12.0,2024-07-01 23:06:04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3194,https://career.habr.com/vacancies/1000144598,2024-06-26 11:03:21,Ведущий программист 1C,NaN,250000.0,₽,Программист 1С,Старший (Senior),Разработка под 1С,NaN,Полный рабочий день,True,"~~~ О компании и команде\nКрафтье Компания """" ...",Bullevie,https://bullevie.ru/,NaN,"Россия, Санкт-Петербург, гп.Федоровское, ул.Пр...",0.0,2024-07-01 23:23:50
3195,https://career.habr.com/vacancies/1000143341,2024-06-25 17:01:12,Software Architect (NGFW),NaN,NaN,NaN,Архитектор программного обеспечения,NaN,Разработка программного обеспечения / Проектир...,NaN,NaN,True,~~~ О компании и команде\nNext Generation Fire...,Лаборатория Касперского,https://kaspersky.ru/,4.20,"Россия, Москва, Ленинградское ш., д. 39А, стр....",7.0,2024-07-01 23:23:50
3196,https://career.habr.com/vacancies/1000144180,2024-06-28 14:36:00,Data Center Infrastructure Engineer / Инженер ...,NaN,NaN,NaN,Системный администратор,Средний (Middle),Администрирование Linux / Bash / Osi model,Санкт-Петербург,Полный рабочий день,NaN,~~~ О компании и команде\nYADRO — это российск...,YADRO,https://yadro.com,NaN,"Россия, Москва, ул. Рочдельская, д. 15, стр. 1...",5.0,2024-07-01 23:23:50
3197,https://career.habr.com/vacancies/1000141715,2024-07-01 12:23:49,Ведущий инженер OpenStack,NaN,NaN,NaN,Инженер по доступности сервисов,Старший (Senior),Linux / Ansible / Terraform,Москва,Полный рабочий день,NaN,~~~ О компании и команде\nМы привлекаем амбици...,Cloud.ru,https://cloud.ru,4.75,"Россия, Москва, ул. 2-я Звенигородская, д. 28",4.0,2024-07-01 23:23:51


In [4]:
df_original.isna().sum()

Ссылка на объявление         0
Дата размещения              0
Название                     0
Зарплата от               2667
Зарплата до               2818
Валюта                    2543
Вакансия                     0
Стаж                       756
Требования                   0
Город                     1075
Тип занятости              339
Можно удаленно            1194
Описание                     0
Работодатель                94
Ссылка на работодателя      94
Рейтинг                   1217
Офисы                      406
Число сотрудников           94
Дата сбора данных            0
dtype: int64

In [5]:
valide_cols = [
    'Дата размещения',
    'Вакансия',
    'Стаж',
    'Зарплата от',
    'Зарплата до',
    'Город',
    'Дата сбора данных',
    'Число сотрудников',
    'Тип занятости',
    'Можно удаленно',
]
def get_valide_cols(dataset, cols):
    return dataset[cols]

In [6]:
exchange_rate_template = {
    "₽": 1,
    "$": 92.37,
    "€": 99.53,
    "₸": 0.206
}
def apply_exchange_rate(row, template=exchange_rate_template):
    row["Зарплата от"] = row["Зарплата от"] * exchange_rate_template[row["Валюта"]] if pd.notna(row["Зарплата от"]) else np.nan
    row["Зарплата до"] = row["Зарплата до"] * exchange_rate_template[row["Валюта"]] if pd.notna(row["Зарплата до"]) else np.nan
    return row

In [7]:
def get_difference_percentiles(df):
    filtered_df = df.dropna(subset=['Зарплата от', 'Зарплата до']).copy()
    filtered_df['Разница'] = filtered_df['Зарплата до'] - filtered_df['Зарплата от']
    condition = filtered_df['Разница'] > 0.8 * filtered_df['Зарплата до']
    filtered_df.loc[condition, ['Зарплата от', 'Разница']] = np.nan

    filtered_df = filtered_df.dropna(subset=['Зарплата от', 'Зарплата до'])

    filtered_df = filtered_df[filtered_df['Зарплата от'] >= 1000]
    filtered_df = filtered_df[filtered_df['Зарплата до'] >= 10000]

    filtered_df['Коэффициент разницы'] = np.where(filtered_df['Зарплата до'] != 0,
                                                      filtered_df['Разница'] / filtered_df['Зарплата до'],
                                                      np.nan)

    perc25 = filtered_df['Коэффициент разницы'].quantile(0.25)
    perc50 = filtered_df['Коэффициент разницы'].quantile(0.50)
    perc75 = filtered_df['Коэффициент разницы'].quantile(0.75)

    return perc25, perc50, perc75


def fill_na_salary(df, coef):
    result_df = df.copy()
    result_df['Зарплата до'] = result_df['Зарплата до'].fillna(result_df['Зарплата от'] / (1-coef))
    result_df['Зарплата от'] = result_df['Зарплата от'].fillna(result_df['Зарплата до'] * (1-coef))
    return result_df

In [8]:
import re

def separate_cities(df):
  work_df = df.copy()
  df_new = pd.DataFrame()
  for index in range(work_df.shape[0]):
    row = re.sub("[\(\[].*?[\)\]]", "", work_df['Город'].iloc[index])
    cities = [city.lstrip().rstrip() for city in row.split(',')]
    init_row = work_df.iloc[index].copy()
    for city in cities:
      new_row = init_row.copy()
      new_row['Город'] = city
      df_new = pd.concat([df_new, pd.DataFrame([new_row])], ignore_index=True)
  return df_new

def find_cities_around_threshold(df, threshold=75):
  cities = df['Город'].unique()
  ratios = {}
  for city in cities:
    ratios[city] = df['Город'].value_counts()[city]/df.shape[0]

  answer = []
  all_percentage = 0
  sorted_dict = {k: v for k, v in sorted(ratios.items(), key=lambda item: item[1], reverse=True)}
  for label, pct in sorted_dict.items():
      all_percentage += pct*100
      if all_percentage < threshold:
          answer.append(label)
  return answer

def create_new_cities(df, cities):
  df_new = df.copy()
  for index in range(df_new.shape[0]):
    if (df_new.iloc[index]['Город'] not in cities):
      df_new.at[index, 'Город'] = 'Малый город'
  return df_new

In [9]:
def fillna_experience_with_vacancy(df):
    dataset = df.copy()

    mean_wages = dataset.groupby(['Вакансия', 'Стаж']).agg({'Зарплата от': 'mean', 'Зарплата до': 'mean'}).reset_index()
    mean_wages['Средняя зарплата'] = (mean_wages['Зарплата от'] + mean_wages['Зарплата до']) / 2

    for i, row in dataset.iterrows():
        if pd.isna(row['Стаж']):
            min_wage = row['Зарплата от']
            max_wage = row['Зарплата до']
            avg_wage = (min_wage + max_wage) / 2
            vacancy_name = row['Вакансия']

            closest_experience = None
            smallest_diff = float('inf')
            for _, r in mean_wages[mean_wages['Вакансия'] == vacancy_name].iterrows():
                wage_diff = abs(avg_wage - r['Средняя зарплата'])
                if wage_diff < smallest_diff:
                    smallest_diff = wage_diff
                    closest_experience = r['Стаж']

            dataset.loc[i, "Стаж"] = closest_experience

    return dataset


def fillna_experience_with_wages(df):
    dataset = df.copy()

    mean_wages = dataset.groupby(['Стаж']).agg({'Зарплата от': 'mean', 'Зарплата до': 'mean'}).reset_index()
    mean_wages['Средняя зарплата'] = (mean_wages['Зарплата от'] + mean_wages['Зарплата до']) / 2

    for i, row in dataset.iterrows():
        if pd.isna(row['Стаж']):
            min_wage = row['Зарплата от']
            max_wage = row['Зарплата до']
            avg_wage = (min_wage + max_wage) / 2

            closest_experience = None
            smallest_diff = float('inf')
            for _, r in mean_wages.iterrows():
                wage_diff = abs(avg_wage - r['Средняя зарплата'])
                if wage_diff < smallest_diff:
                    smallest_diff = wage_diff
                    closest_experience = r['Стаж']

            dataset.loc[i, "Стаж"] = closest_experience

    return dataset

Удаляем строки, в которых одновременно отсутствует информация о зарплате в обоих столбцах. Заменяем пропуски нулями и делаем столбец категориальным. Заполняем столбец "Город"

In [10]:
def preprocessing_data(df):
   df_new = df.copy()

   missing_data = df_new[df["Зарплата от"].isnull() & df_new["Зарплата до"].isnull()]
   df_new = df_new.drop(missing_data.index)

   df_new['Можно удаленно'].fillna(0, inplace=True)
   df_new['Можно удаленно'] = df_new['Можно удаленно'].astype('int64')
   df_new['Можно удаленно'] = df_new['Можно удаленно'].apply(lambda x: "Да" if x == 1 else "Нет")

   df_new.loc[(df_new['Можно удаленно'] == "Нет") & (df_new['Город'].isnull()), 'Город'] = 'Не указано'
   df_new.loc[(df_new['Можно удаленно'] == "Да") & (df_new['Город'].isnull()), 'Город'] = 'Удаленно'

   df_new['Тип занятости'].fillna('Не указан', inplace=True)

   df_new['Число сотрудников'].fillna(0, inplace=True)
   df_new['Число сотрудников'] = df_new['Число сотрудников'].astype('int64')

   df_new = df_new.apply(apply_exchange_rate, axis=1)
   df_new.drop("Валюта", axis=1, inplace=True)

   perc25, perc50, perc75 = get_difference_percentiles(df_new)
   df_filled_salary = fill_na_salary(df_new, perc50)

   # df_exp = fillna_experience_with_vacancy(df_25)

   df_result = get_valide_cols(df_filled_salary, valide_cols)
   return df_result


def sep_cities_data(df):
    df_new = df.copy()

    df_new = separate_cities(df_new)
    # cities = find_cities_around_threshold(df_new, 85)
    # df_new = create_new_cities(df_new, cities)

    return df_new

In [12]:
df_prep = preprocessing_data(df_original)
# df_exp_vac = fillna_experience_with_vacancy(df_prep)
df_exp = fillna_experience_with_wages(df_prep) # fillna_experience_with_wages(df_exp_vac)
df_sep_cities = sep_cities_data(df_exp)

In [13]:
def analysis_missing_values_vacancy(df):
    data = df.copy()

    data['Наименьшая зарплата'] = data['Зарплата от']
    data['Наибольшая зарплата'] = data['Зарплата до']

    vacancy_missing_values = data['Стаж'].isnull().groupby(data['Вакансия']).mean()
    wages_values = data.groupby(['Вакансия', 'Стаж']).agg({'Зарплата от': 'mean',
                                                           'Зарплата до': 'mean',
                                                           'Наименьшая зарплата': 'min',
                                                           'Наибольшая зарплата': 'max'})

    return vacancy_missing_values[vacancy_missing_values > 0.15], wages_values

In [14]:
df_analysis, df_wages = analysis_missing_values_vacancy(df_prep)
df_analysis.sort_values()

Вакансия
Инженер технической поддержки          0.153846
ML разработчик                         0.166667
Архитектор программного обеспечения    0.166667
Менеджер по найму                      0.166667
Системный инженер                      0.166667
Бэкенд разработчик                     0.202614
Инженер по ручному тестированию        0.210526
Менеджер технической поддержки         0.250000
Инженер по производительности          0.333333
Инженер по компьютерному зрению        0.333333
Разработчик приложений                 0.400000
Менеджер по маркетингу                 0.500000
ERP-программист                        0.500000
Директор проекта                       0.500000
Технический директор                   0.500000
Директор по маркетингу                 1.000000
Менеджер по обеспечению качества       1.000000
Антифрод аналитик                      1.000000
Администратор сайта                    1.000000
Name: Стаж, dtype: float64

In [15]:
# from matplotlib import pyplot as plt

# plt.figure(figsize=(8, 8))
# plt.pie(df_analysis, labels=df_analysis.index, autopct='%1.1f%%', startangle=140)
# plt.axis('equal')  # Равные пропорции для круговой диаграммы
# plt.title('Доля пропущенных значений стажа по вакансиям')
# plt.show()

In [16]:
df_wages

Зарплата от    Зарплата до  \
Вакансия            Стаж                                             
DevOps-инженер      Младший (Junior)   93333.333333  171666.666667   
                    Средний (Middle)  201224.035088  302625.526316   
                    Стажёр (Intern)    40000.000000   60000.000000   
                    Старший (Senior)  366777.500000  505198.750000   
ERP-программист     Средний (Middle)  150000.000000  200000.000000   
...                                             ...            ...   
Фулстек разработчик Младший (Junior)   50000.000000  120000.000000   
                    Средний (Middle)  146406.877193  222743.868421   
                    Стажёр (Intern)    40000.000000   60000.000000   
                    Старший (Senior)  323355.000000  478573.277778   
Юрист               Старший (Senior)  100000.000000  150000.000000   

                                      Наименьшая зарплата  Наибольшая зарплата  
Вакансия            Стаж                                                        
DevOps-инженер      Младший (Junior)              60000.0             250000.0  
                    Средний (Middle)              50000.0             692775.0  
                    Стажёр (Intern)               40000.0              60000.0  
                    Старший (Senior)             250000.0             646590.0  
ERP-программист     Средний (Middle)             150000.0             200000.0  
...                                                   ...                  ...  
Фулстек разработчик Младший (Junior)              50000.0             120000.0  
                    Средний (Middle)              40000.0             450000.0  
                    Стажёр (Intern)               40000.0              60000.0  
                    Старший (Senior)             150000.0             671827.5  
Юрист               Старший (Senior)             100000.0             150000.0  

[153 rows x 4 columns]

In [17]:
df_analysis_uniq_exp = df_prep.groupby(['Вакансия'])['Стаж'].agg(['unique'])
df_analysis_uniq_exp

,unique
Вакансия,
DevOps-инженер,"[Старший (Senior), Средний (Middle), nan, Млад..."
ERP-программист,"[Средний (Middle), nan]"
HTML-верстальщик,[Средний (Middle)]
ML разработчик,"[Средний (Middle), nan, Стажёр (Intern), Старш..."
PR-менеджер,[Стажёр (Intern)]
...,...
Технический писатель,"[Средний (Middle), Старший (Senior), Стажёр (I..."
Ученый по данным,"[Стажёр (Intern), Старший (Senior)]"
Фронтенд разработчик,"[Средний (Middle), Младший (Junior), Старший (..."


In [18]:
filtered_df = df_analysis_uniq_exp[df_analysis_uniq_exp.apply(lambda x : len(x['unique']) <= 1, axis=1)]
filtered_df

,unique
Вакансия,
HTML-верстальщик,[Средний (Middle)]
PR-менеджер,[Стажёр (Intern)]
SMM-специалист,[Стажёр (Intern)]
Администратор защиты,[Средний (Middle)]
Администратор сайта,[nan]
Аналитик мобильных приложений,[Средний (Middle)]
Аналитик по обеспечению качества,[Средний (Middle)]
Аналитик продаж,[Средний (Middle)]
Аналитик технической поддержки,[Средний (Middle)]


In [19]:
filtered_df.shape

(25, 1)

In [20]:
df_anal = df_exp.copy()
df_anal['Наименьшая зарплата'] = df_anal['Зарплата от']
df_anal['Наибольшая зарплата'] = df_anal['Зарплата до']
df_anal.groupby(['Стаж']).agg({
    'Зарплата от': 'mean',
    'Зарплата до': 'mean',
    'Наименьшая зарплата': 'min',
    'Наибольшая зарплата': 'max'
})

,Зарплата от,Зарплата до,Наименьшая зарплата,Наибольшая зарплата
Стаж,,,,
Младший (Junior),75377.235507,120854.478261,36000.0,250000.0
Средний (Middle),155139.248036,230610.065657,38000.0,692775.0
Стажёр (Intern),44490.862069,70429.137931,5000.0,369480.0
Старший (Senior),272772.456853,409829.121827,92370.0,1492950.0


In [21]:
df_prep[df_prep['Вакансия'] == 'Инженер по производительности']

,Дата размещения,Вакансия,Стаж,Зарплата от,Зарплата до,Город,Дата сбора данных,Число сотрудников,Тип занятости,Можно удаленно
469,2024-07-01 12:27:12,Инженер по производительности,NaN,166666.666667,250000.0,Удаленно,2024-07-01 23:08:37,10,Полный рабочий день,Да
516,2024-07-01 19:27:15,Инженер по производительности,Старший (Senior),230000.000000,345000.0,Москва,2024-07-01 23:08:53,83,Полный рабочий день,Нет
2981,2024-06-28 11:42:14,Инженер по производительности,Старший (Senior),233333.333333,350000.0,Москва,2024-07-01 23:22:39,6,Полный рабочий день,Нет


In [22]:
df_exp[df_exp['Вакансия'] == 'Инженер по производительности']

,Дата размещения,Вакансия,Стаж,Зарплата от,Зарплата до,Город,Дата сбора данных,Число сотрудников,Тип занятости,Можно удаленно
469,2024-07-01 12:27:12,Инженер по производительности,Средний (Middle),166666.666667,250000.0,Удаленно,2024-07-01 23:08:37,10,Полный рабочий день,Да
516,2024-07-01 19:27:15,Инженер по производительности,Старший (Senior),230000.000000,345000.0,Москва,2024-07-01 23:08:53,83,Полный рабочий день,Нет
2981,2024-06-28 11:42:14,Инженер по производительности,Старший (Senior),233333.333333,350000.0,Москва,2024-07-01 23:22:39,6,Полный рабочий день,Нет


In [23]:
df_exp

,Дата размещения,Вакансия,Стаж,Зарплата от,Зарплата до,Город,Дата сбора данных,Число сотрудников,Тип занятости,Можно удаленно
0,2024-06-27 16:50:48,Архитектор программного обеспечения,Старший (Senior),333333.333333,500000.0,Москва,2024-07-01 23:06:03,0,Полный рабочий день,Нет
2,2024-06-04 16:24:59,Менеджер проекта,Стажёр (Intern),5000.000000,25000.0,"Москва, Санкт-Петербург, Казань",2024-07-01 23:06:04,1,Полный рабочий день,Да
3,2024-07-01 12:09:30,Бэкенд разработчик,Средний (Middle),210000.000000,315000.0,Удаленно,2024-07-01 23:06:04,0,Полный рабочий день,Да
8,2024-06-04 09:23:18,Менеджер по найму,Стажёр (Intern),66666.666667,100000.0,Удаленно,2024-07-01 23:06:06,0,Полный рабочий день,Да
18,2024-06-06 13:34:21,Менеджер технической поддержки,Средний (Middle),60000.000000,90000.0,Удаленно,2024-07-01 23:06:10,0,Полный рабочий день,Да
...,...,...,...,...,...,...,...,...,...,...
3153,2024-07-01 13:29:52,Веб-разработчик,Средний (Middle),200000.000000,300000.0,Москва,2024-07-01 23:23:36,0,Полный рабочий день,Нет
3160,2024-06-03 15:46:09,Фулстек разработчик,Стажёр (Intern),40000.000000,60000.0,Москва,2024-07-01 23:23:39,1,Неполный рабочий день,Да
3176,2024-06-26 13:17:01,DevOps-инженер,Старший (Senior),400000.000000,600000.0,Удаленно,2024-07-01 23:23:44,0,Полный рабочий день,Да
3181,2024-06-27 10:06:33,Фулстек разработчик,Старший (Senior),300000.000000,450000.0,Екатеринбург,2024-07-01 23:23:45,0,Полный рабочий день,Нет


In [24]:
df_sep_cities

,Дата размещения,Вакансия,Стаж,Зарплата от,Зарплата до,Город,Дата сбора данных,Число сотрудников,Тип занятости,Можно удаленно
0,2024-06-27 16:50:48,Архитектор программного обеспечения,Старший (Senior),333333.333333,500000.0,Москва,2024-07-01 23:06:03,0,Полный рабочий день,Нет
1,2024-06-04 16:24:59,Менеджер проекта,Стажёр (Intern),5000.000000,25000.0,Москва,2024-07-01 23:06:04,1,Полный рабочий день,Да
2,2024-06-04 16:24:59,Менеджер проекта,Стажёр (Intern),5000.000000,25000.0,Санкт-Петербург,2024-07-01 23:06:04,1,Полный рабочий день,Да
3,2024-06-04 16:24:59,Менеджер проекта,Стажёр (Intern),5000.000000,25000.0,Казань,2024-07-01 23:06:04,1,Полный рабочий день,Да
4,2024-07-01 12:09:30,Бэкенд разработчик,Средний (Middle),210000.000000,315000.0,Удаленно,2024-07-01 23:06:04,0,Полный рабочий день,Да
...,...,...,...,...,...,...,...,...,...,...
761,2024-07-01 13:29:52,Веб-разработчик,Средний (Middle),200000.000000,300000.0,Москва,2024-07-01 23:23:36,0,Полный рабочий день,Нет
762,2024-06-03 15:46:09,Фулстек разработчик,Стажёр (Intern),40000.000000,60000.0,Москва,2024-07-01 23:23:39,1,Неполный рабочий день,Да
763,2024-06-26 13:17:01,DevOps-инженер,Старший (Senior),400000.000000,600000.0,Удаленно,2024-07-01 23:23:44,0,Полный рабочий день,Да
764,2024-06-27 10:06:33,Фулстек разработчик,Старший (Senior),300000.000000,450000.0,Екатеринбург,2024-07-01 23:23:45,0,Полный рабочий день,Нет


In [25]:
df_exp.isna().sum()

Дата размещения      0
Вакансия             0
Стаж                 0
Зарплата от          0
Зарплата до          0
Город                0
Дата сбора данных    0
Число сотрудников    0
Тип занятости        0
Можно удаленно       0
dtype: int64

In [26]:
df_sep_cities.isna().sum()

Дата размещения      0
Вакансия             0
Стаж                 0
Зарплата от          0
Зарплата до          0
Город                0
Дата сбора данных    0
Число сотрудников    0
Тип занятости        0
Можно удаленно       0
dtype: int64

In [27]:
df_sep_cities['Город'].unique()

array(['Москва', 'Санкт-Петербург', 'Казань', 'Удаленно', 'Новый Уренгой',
       'Краснодар', 'Омск', 'Самара', 'Екатеринбург', 'Ростов-на-Дону',
       'Тюмень', 'Нижний Новгород', 'Липецк', 'Барнаул', 'Новосибирск',
       'Томск', 'Стерлитамак', 'Токио', 'Набережные Челны', 'Ставрополь',
       'Псков', 'Лимассол', 'Пенза', 'Саратов', 'Уфа', 'Кемерово',
       'Севастополь', 'Челябинск', 'Воронеж', 'Пушкино', 'Рязань',
       'Брянск', 'Хошимин', 'Ханой', 'Дананг', 'Пермь', 'Кострома',
       'Коломна', 'Таллин', 'Хабаровск', 'Калининград', 'Владивосток',
       'Тель-Авив', 'Красноярск', 'Симферополь', 'Астрахань', 'Иркутск',
       'Ханты-Мансийск', 'Сочи', 'Ижевск', 'Тольятти', 'Энгельс',
       'Майкоп', 'Химки', 'Бердск', 'Курган', 'Ярославль', 'Балашиха',
       'Зеленоград', 'Чебоксары', 'Ульяновск', 'Лос-Анджелес', 'Салехард',
       'Минск', 'Дубай', 'Донецк', 'Тула', 'Иваново', 'Комсомольск'],
      dtype=object)

In [28]:
# Эспорт общей и городской базы данных
df_exp.to_excel("habr_general.xlsx")
df_sep_cities.to_excel("habr_multicity.xlsx")

In [29]:
df_outliers = df_exp[['Стаж', 'Зарплата от', 'Зарплата до']]
df_outliers

,Стаж,Зарплата от,Зарплата до
0,Старший (Senior),333333.333333,500000.0
2,Стажёр (Intern),5000.000000,25000.0
3,Средний (Middle),210000.000000,315000.0
8,Стажёр (Intern),66666.666667,100000.0
18,Средний (Middle),60000.000000,90000.0
...,...,...,...
3153,Средний (Middle),200000.000000,300000.0
3160,Стажёр (Intern),40000.000000,60000.0
3176,Старший (Senior),400000.000000,600000.0
3181,Старший (Senior),300000.000000,450000.0


In [30]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import mahalanobis
from sklearn.preprocessing import LabelEncoder
from scipy.stats import chi2

data = df_outliers.copy()

label_encoder = LabelEncoder()
data['category_encoded'] = label_encoder.fit_transform(data['Стаж'])

# data['Средняя зарплата'] = (data['Зарплата от'] + data['Зарплата до'])/2
numerical_data = data[['Зарплата от', 'Зарплата до']]
mean = numerical_data.mean(axis=0)
cov_matrix = np.cov(numerical_data.values.T)
inv_cov_matrix = np.linalg.inv(cov_matrix)

def mahalanobis_distance(row, mean, inv_cov_matrix):
    return mahalanobis(row, mean, inv_cov_matrix)

numerical_data['mahalanobis'] = numerical_data.apply(lambda row: mahalanobis_distance(row, mean, inv_cov_matrix), axis=1)

def hamming_distance(row, reference):
    return row != reference

reference_category = data['category_encoded'].mode()[0]

data['categorical_distance'] = data['category_encoded'].apply(lambda x: hamming_distance(x, reference_category))

weight_numerical = 0.5
weight_categorical = 0.5

data['combined_distance'] = (weight_numerical * numerical_data['mahalanobis']) + (weight_categorical * data['categorical_distance'])

combined_threshold = data['combined_distance'].quantile(0.80)
data['outlier'] = data['combined_distance'] > combined_threshold

print(data)

                  Стаж    Зарплата от  Зарплата до  category_encoded  \
0     Старший (Senior)  333333.333333     500000.0                 3   
2      Стажёр (Intern)    5000.000000      25000.0                 2   
3     Средний (Middle)  210000.000000     315000.0                 1   
8      Стажёр (Intern)   66666.666667     100000.0                 2   
18    Средний (Middle)   60000.000000      90000.0                 1   
...                ...            ...          ...               ...   
3153  Средний (Middle)  200000.000000     300000.0                 1   
3160   Стажёр (Intern)   40000.000000      60000.0                 2   
3176  Старший (Senior)  400000.000000     600000.0                 3   
3181  Старший (Senior)  300000.000000     450000.0                 3   
3194  Старший (Senior)  166666.666667     250000.0                 3   

      categorical_distance  combined_distance  outlier  
0                     True           1.267493     True  
2                    

<ipython-input-30-e6a1b33b0123>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numerical_data['mahalanobis'] = numerical_data.apply(lambda row: mahalanobis_distance(row, mean, inv_cov_matrix), axis=1)


In [31]:
data[data['outlier'] == True]['Стаж'].value_counts()

Стаж
Старший (Senior)    74
Стажёр (Intern)     35
Средний (Middle)     8
Младший (Junior)     7
Name: count, dtype: int64

In [32]:
data[data['outlier'] == True]

,Стаж,Зарплата от,Зарплата до,category_encoded,categorical_distance,combined_distance,outlier
0,Старший (Senior),333333.333333,500000.0,3,True,1.267493,True
2,Стажёр (Intern),5000.000000,25000.0,2,True,1.221361,True
48,Старший (Senior),250000.000000,600000.0,3,True,3.004273,True
65,Стажёр (Intern),25000.000000,37500.0,2,True,1.142541,True
66,Старший (Senior),646590.000000,831330.0,3,True,2.913311,True
...,...,...,...,...,...,...,...
3052,Старший (Senior),323295.000000,484942.5,3,True,1.221589,True
3065,Старший (Senior),350000.000000,525000.0,3,True,1.343708,True
3084,Старший (Senior),461850.000000,692775.0,3,True,1.855191,True
3121,Старший (Senior),280000.000000,350000.0,3,True,1.333329,True


In [33]:
df_cleaned = df_exp.drop(data.index[data['outlier']])
df_cleaned

,Дата размещения,Вакансия,Стаж,Зарплата от,Зарплата до,Город,Дата сбора данных,Число сотрудников,Тип занятости,Можно удаленно
3,2024-07-01 12:09:30,Бэкенд разработчик,Средний (Middle),210000.000000,315000.0,Удаленно,2024-07-01 23:06:04,0,Полный рабочий день,Да
8,2024-06-04 09:23:18,Менеджер по найму,Стажёр (Intern),66666.666667,100000.0,Удаленно,2024-07-01 23:06:06,0,Полный рабочий день,Да
18,2024-06-06 13:34:21,Менеджер технической поддержки,Средний (Middle),60000.000000,90000.0,Удаленно,2024-07-01 23:06:10,0,Полный рабочий день,Да
23,2024-06-27 09:27:06,Системный инженер,Старший (Senior),233333.333333,350000.0,Новый Уренгой,2024-07-01 23:06:11,10,Полный рабочий день,Нет
38,2024-07-01 13:29:52,Системный администратор,Средний (Middle),130000.000000,195000.0,Москва,2024-07-01 23:06:17,0,Полный рабочий день,Нет
...,...,...,...,...,...,...,...,...,...,...
3143,2024-06-27 16:30:25,Администратор баз данных,Средний (Middle),266666.666667,400000.0,Москва,2024-07-01 23:23:32,6,Полный рабочий день,Нет
3153,2024-07-01 13:29:52,Веб-разработчик,Средний (Middle),200000.000000,300000.0,Москва,2024-07-01 23:23:36,0,Полный рабочий день,Нет
3160,2024-06-03 15:46:09,Фулстек разработчик,Стажёр (Intern),40000.000000,60000.0,Москва,2024-07-01 23:23:39,1,Неполный рабочий день,Да
3181,2024-06-27 10:06:33,Фулстек разработчик,Старший (Senior),300000.000000,450000.0,Екатеринбург,2024-07-01 23:23:45,0,Полный рабочий день,Нет


In [34]:
df_cleaned['Наименьшая зарплата'] = df_cleaned['Зарплата от']
df_cleaned['Наибольшая зарплата'] = df_cleaned['Зарплата до']
df_cleaned.groupby(['Стаж']).agg({
    'Зарплата от': 'mean',
    'Зарплата до': 'mean',
    'Наименьшая зарплата': 'min',
    'Наибольшая зарплата': 'max'
})

,Зарплата от,Зарплата до,Наименьшая зарплата,Наибольшая зарплата
Стаж,,,,
Младший (Junior),79111.611111,117879.641026,36000.0,210000.0
Средний (Middle),152755.957324,226117.679931,38000.0,554220.0
Стажёр (Intern),52348.148148,79664.197531,30000.0,225000.0
Старший (Senior),220538.008130,331151.483740,92370.0,450000.0
